In [ ]:
# import dependencies
import datetime as dt
import pandas as pd
import numpy as np
import requests, io, zipfile, os, time
import dask.dataframe as dd

In [ ]:
# read in full csv and scale for analysis

# for curiosity, time taken to read data
## dask documentation from https://www.geeksforgeeks.org/working-with-large-csv-files-in-python/
s_time = time.time()

# Read CSV files from List
ddf = dd.read_csv("full_raw_data.csv",
            low_memory=False,
            dtype={
                    'end_station_id': 'object',
                    'start_station_id': 'object'
                  },
                 )

# end of data read, print time report
e_time = time.time()
print("Processed in ", ((e_time-s_time)/60), "minutes")

ddf.dtypes

In [ ]:
# dask compute the DataFrame values
##  to create the pandas DataFrame

s_time = time.time()
df = ddf.compute()
e_time = time.time()
print("Processed in ", ((e_time-s_time)/60), "minutes")

In [ ]:
# # create pandas DataFrame

# s_time = time.time()
# df = pd.DataFrame(ddf)
# e_time = time.time()
# print("Processed in ", ((e_time-s_time)/60), "minutes")

In [ ]:
print(f'df contains {df.shape[0]} rows and {df.shape[1]} columns.')
print('~' * 50)
df.dtypes

In [ ]:
# set up date formats and rename columns

##  start date/time
df["started_at"] = df["started_at"].astype('datetime64[ns]')
df['started_year'] = df['started_at'].dt.year
df['started_month'] = df['started_at'].dt.month_name
df['started_weekday'] = df['started_at'].dt.day_name
df['started_hour'] = df['started_at'].dt.hour


##  end date/time
df["ended_at"] = df["ended_at"].astype('datetime64[ns]')

## add column for ride time
df['ride_minutes'] = df["ended_at"] - df["started_at"]

# rename columns
df = df.rename(columns={
                        'start_station_name' : 'start_station',
                        'end_station_name': 'end_station',
                        'member_casual' : 'member_type'
                        })

In [ ]:
df.dtypes

In [ ]:
# select columns for final csv file

df_cleaned = pd.DataFrame(df, columns={
                                    'start_station_name',    
                                    'end_station_name',
                                    'started_year',
                                    'started_month',
                                    'started_hour',
                                    'ride_minutes',
                                    'rideable_type',
                                    'member_type'    
                                    })

In [ ]:
df.info()

In [ ]:
# send end_df rides to csv file
path = f'{root}/clean_data.csv'
end_df.to_csv(path)